In [2]:
import math
import numpy as np
import pandas as pd
import csv
from tqdm import tqdm

print('环境创建成功')

环境创建成功


In [2]:
class dotPair(object):  # 点对，用于表示组成三角形的三个线段
    abc = []
    def __init__(self, sPoint=None, ePoint=None):
        self.sPoint = sPoint
        self.ePoint = ePoint
        self.sp = [self.sPoint[0], self.sPoint[1]]
        self.ep = [self.ePoint[0], self.ePoint[1]]
        self.abc = [self.sp[1] - self.ep[1], self.ep[0] - self.sp[0], np.linalg.det([self.sp, self.ep])]

In [3]:
class line(object):  # 射线类，具有起始点以及方向，用于表示每个三角形上水流的起始点以及流动的方向
    sPoint = []
    theta = []
    alpha = []
    abc = []

    def __init__(self, sPoint=None, alpha=None):  # 输入的alpha为坡向
        self.sPoint = sPoint
        if alpha[0] <= math.pi / 2:   
            theta = math.pi / 2 - alpha[0]      #此处的theta为直线的倾斜角
        elif math.pi / 2 < alpha[0] <= math.pi * 3 / 2:
            theta = math.pi * 3 / 2 - alpha[0]
        else:
            theta = math.pi * 5 / 2 - alpha[0]
        self.theta = [round(np.cos(theta), 6), round(np.sin(theta), 6)]   # 计算倾斜角的余弦正弦值，用于求射线的一般式方程的abc系数
        self.alpha = [round(np.sin(alpha[0]), 6), round(np.cos(alpha[0]), 6)]  # 计算坡向的正弦余弦值，用于判别交点是否在射线方向上
        self.sp = [self.sPoint[0], self.sPoint[1]]
        self.abc = [self.theta[1], -self.theta[0], np.linalg.det([self.theta, self.sp])]

In [4]:
def ifIntersect(line1, line2):  # 判断射线与线段是否有交点，如果有交点则返回1。
    sFlag = np.sum(np.array(line1.abc) * np.array([line2.sPoint[0], line2.sPoint[1], 1]))
    eFlag = np.sum(np.array(line1.abc) * np.array([line2.ePoint[0], line2.ePoint[1], 1]))
    if round(sFlag * eFlag) <= 0:   # 有交点的话，将线段的两个端点带入射线的方程ax+by+c，得出的结果一定是异号。
        num = 1
    else:
        num = 0
    return num

In [5]:
def getDistance(ray, dp):  # l1是射线，l2是线段
#     Dotabc = [dp.sPoint[1] - dp.ePoint[1], dp.ePoint[0] - dp.sPoint[0], np.linalg.det([dp.sp, dp.ep])]  # 计算线段的一般式方程的abc三个系数
    delta = ray.abc[0] * dp.abc[1] - dp.abc[0] * ray.abc[1]
    if delta == 0:
        return None
    c1 = (ray.abc[1] * dp.abc[2] - dp.abc[1] * ray.abc[2]) / delta
    c2 = (dp.abc[0] * ray.abc[2] - ray.abc[0] * dp.abc[2]) / delta
    cross = [round(c1, 2), round(c2, 2)]  # 利用判别式求射线与线段的交点坐标
    distance = math.sqrt((ray.sPoint[0] - cross[0]) ** 2 + (ray.sPoint[1] - cross[1]) ** 2)
    if cross == ray.sp or distance < 0.005:   # 如果交点恰好是射线的起点，则返回None，否则后续迭代计算时，若第一步的线段恰好是上一步交点所在的线段，判断会停止
        return None
    d1 = (cross[0] - ray.sPoint[0]) * ray.alpha[0]   # 判断交点是否在射线方向上。若在射线方向上，则交点和始点的x、y的差值分别与坡向的正弦、余弦三角函数同号
    d2 = (cross[1] - ray.sPoint[1]) * ray.alpha[1]
    d3 = ifIntersect(ray, dp)
    if d1 >= 0 and d2 >= 0 and d3 == 1:
#         distance = math.sqrt((ray.sPoint[0] - cross[0]) ** 2 + (ray.sPoint[1] - cross[1]) ** 2)
        return cross, round(distance, 6)

In [6]:
line_seg = dotPair([375432.3359,3176777.1115], [375432.3359,3176747.1115])
ray = line(sPoint=[375452.588633,3176756.859269], alpha=[198.434949/180*math.pi])
print(ifIntersect(ray, line_seg))
print(getDistance(ray, line_seg))

0
None


In [7]:
line_seg = dotPair([0,1],[2,1])
ray = line(sPoint=[1,1],alpha=[math.pi/4])
print(ifIntersect(ray, line_seg))
print(getDistance(ray, line_seg))

1
None


# 数据清洗

In [8]:
tn = pd.read_csv("D:/edu/研究生数据/30mdata/table_resolutions/30m/30mspacejoin.csv")
for i in tn.index:
    if tn.iloc[i,1] < 180:
        tn.iloc[i,1] = (tn.iloc[i,1] + 180)*math.pi/180
    else:
        tn.iloc[i,1] = (tn.iloc[i,1] - 180)*math.pi/180
tn

,Slope_Pct,Aspect,Tri_Index,Node_Index,TinNod.xyz
0,7.453560,5.176037,67694,55062,"316065.22,2899853.82,197"
1,7.453560,5.176037,67694,55063,"316095.22,2899853.82,195"
2,7.453560,5.176037,67694,7,"316095.22,2899883.82,196"
3,24.267033,3.419892,68267,55062,"316065.22,2899853.82,197"
4,24.267033,3.419892,68267,55063,"316095.22,2899853.82,195"
...,...,...,...,...,...
690925,23.570226,4.854286,110119,7,"316095.22,2899883.82,196"
690926,23.570226,4.854286,110119,54987,"316125.22,2899883.82,189"
690927,3.333333,4.712389,154330,55062,"316065.22,2899853.82,197"
690928,3.333333,4.712389,154330,7,"316095.22,2899883.82,196"


In [9]:
tin_nodes = tn.groupby(['Tri_Index','Slope_Pct','Aspect','Node_Index']).first()
tin_nodes

TinNod.xyz
Tri_Index Slope_Pct Aspect   Node_Index                          
3         30.731815 5.421015 147         321675.22,2903903.82,356
                             148         321705.22,2903903.82,349
                             1090        321705.22,2903873.82,343
4         6.666667  3.141593 2028        314955.22,2905613.82,223
                             12476       314985.22,2905643.82,221
...                                                           ...
232511    29.814240 2.034444 116233      321795.22,2905133.82,522
                             116259      321795.22,2905148.82,520
232513    26.874192 2.089942 296         321765.22,2905163.82,511
                             116259      321795.22,2905148.82,520
                             116260      321795.22,2905163.82,518

[690930 rows x 1 columns]

In [10]:
temp = tin_nodes.to_dict()['TinNod.xyz']
tndict = {}
temp

{(3, 30.73181486, 5.4210152517268835, 147): '321675.22,2903903.82,356',
 (3, 30.73181486, 5.4210152517268835, 148): '321705.22,2903903.82,349',
 (3, 30.73181486, 5.4210152517268835, 1090): '321705.22,2903873.82,343',
 (4, 6.666666667, 3.141592653589793, 2028): '314955.22,2905613.82,223',
 (4, 6.666666667, 3.141592653589793, 12476): '314985.22,2905643.82,221',
 (4, 6.666666667, 3.141592653589793, 25587): '314985.22,2905613.82,223',
 (8, 6.666666667, 0.0, 53): '313515.22,2904953.82,167',
 (8, 6.666666667, 0.0, 55): '313485.22,2904923.82,165',
 (8, 6.666666667, 0.0, 316): '313485.22,2904953.82,167',
 (10, 30.18461713, 4.601731759139407, 23974): '312975.22,2893793.82,245',
 (10, 30.18461713, 4.601731759139407, 24019): '312975.22,2893733.82,247',
 (10, 30.18461713, 4.601731759139407, 65204): '312945.22,2893763.82,255',
 (14, 23.33333333, 4.71238898038469, 9495): '310485.22,2896253.82,475',
 (14, 23.33333333, 4.71238898038469, 9515): '310455.22,2896223.82,482',
 (14, 23.33333333, 4.712388980

## 字典tndict（Tin-Node Dict)

格式如下：

```python
tndict = {
    '(tri_index_1, slope_Deg, aspect)': 
        {
        'node_index_1': 'coord_x, coord_y，coord_z',
        'node_index_2': 'coord_x, coord_y，coord_z',
        'node_index_3': 'coord_x, coord_y，coord_z'
        },
    ...,
    '(tri_index_n, slope_Deg, aspect)': 
        {
        'node_index_1': 'coord_x, coord_y，coord_z',
        'node_index_2': 'coord_x, coord_y，coord_z',
        'node_index_3': 'coord_x, coord_y，coord_z'
        }
}
```
该字典中，键为一个元组，元组中的元素为三角形编号，该三角形的坡度，坡向；值为一个字典,该字典有三个键值对，键为组成对应值字典的键中三角形的三个节点编号，值为该节点编号的xyz坐标。

In [11]:
for k in temp:
    tid = (k[0],k[1],k[2])
    nid = k[3]
    nodxzy = temp[k]
    if tid not in tndict:
        tndict[tid] = {}
    tndict[tid][nid] = nodxzy
tndict      

{(3, 30.73181486, 5.4210152517268835): {147: '321675.22,2903903.82,356',
  148: '321705.22,2903903.82,349',
  1090: '321705.22,2903873.82,343'},
 (4, 6.666666667, 3.141592653589793): {2028: '314955.22,2905613.82,223',
  12476: '314985.22,2905643.82,221',
  25587: '314985.22,2905613.82,223'},
 (8, 6.666666667, 0.0): {53: '313515.22,2904953.82,167',
  55: '313485.22,2904923.82,165',
  316: '313485.22,2904953.82,167'},
 (10, 30.18461713, 4.601731759139407): {23974: '312975.22,2893793.82,245',
  24019: '312975.22,2893733.82,247',
  65204: '312945.22,2893763.82,255'},
 (14, 23.33333333, 4.71238898038469): {9495: '310485.22,2896253.82,475',
  9515: '310455.22,2896223.82,482',
  20762: '310485.22,2896223.82,475'},
 (15, 3.333333333, 0.0): {17041: '321255.22,2901683.82,200',
  31954: '321225.22,2901713.82,201',
  31979: '321225.22,2901683.82,200'},
 (18, 44.84541349, 2.303611428654416): {49: '321735.22,2905253.82,471',
  292: '321735.22,2905193.82,489',
  802: '321705.22,2905223.82,470'},
 (22

In [12]:
tn

,Slope_Pct,Aspect,Tri_Index,Node_Index,TinNod.xyz
0,7.453560,5.176037,67694,55062,"316065.22,2899853.82,197"
1,7.453560,5.176037,67694,55063,"316095.22,2899853.82,195"
2,7.453560,5.176037,67694,7,"316095.22,2899883.82,196"
3,24.267033,3.419892,68267,55062,"316065.22,2899853.82,197"
4,24.267033,3.419892,68267,55063,"316095.22,2899853.82,195"
...,...,...,...,...,...
690925,23.570226,4.854286,110119,7,"316095.22,2899883.82,196"
690926,23.570226,4.854286,110119,54987,"316125.22,2899883.82,189"
690927,3.333333,4.712389,154330,55062,"316065.22,2899853.82,197"
690928,3.333333,4.712389,154330,7,"316095.22,2899883.82,196"


In [13]:
nodes_tin =tn.groupby(['Node_Index','Tri_Index'])['Slope_Pct','Aspect'].first()
#nodes_tin.index

C:\Users\LZY\AppData\Local\Temp/ipykernel_1960/4093753947.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  nodes_tin =tn.groupby(['Node_Index','Tri_Index'])['Slope_Pct','Aspect'].first()


## 字典ntdict（Node - Tin Dict）

用于查询某个节点属于哪些三角形的组成节点。

In [14]:
ntdict = {}
for i in nodes_tin.index:
    nid = i[0]
    tid = i[1]
    if nid not in ntdict:
        ntdict[nid] = []
    ntdict[nid].append(tid)
ntdict

{5: [13808, 29926, 30255, 56997, 232265, 232333, 232342],
 6: [1089],
 7: [67694, 109968, 110119, 154180, 154330, 191920],
 10: [1413, 232254],
 15: [214],
 16: [8967, 21855],
 17: [22644, 83756],
 18: [16621, 36250],
 19: [48823, 48845],
 20: [491, 9769],
 21: [51133],
 22: [131, 2525],
 23: [191],
 24: [3862, 23787],
 25: [631, 25792, 52371, 232301],
 26: [1596, 1597, 1598],
 27: [27565, 27640, 53885, 54170, 91718, 91910, 135715],
 28: [260, 659, 671, 1847, 1866, 4999, 5002],
 29: [105, 692, 1882, 1932, 5037, 5113, 12415, 12467],
 30: [688, 693, 698, 1928, 1949, 5132],
 31: [293, 776, 778, 2107, 2120, 2132],
 32: [2219, 2249, 14156],
 33: [127, 334, 337, 2513, 2519],
 34: [921, 933, 2561, 6557, 6595, 16171, 16172, 16184],
 35: [28, 1039, 2823, 2825, 2837, 2845, 7365, 7374],
 36: [28, 2837, 2853, 2865, 18295, 18327, 18328],
 37: [1089, 41709],
 38: [3014, 3028, 7808, 7895, 19235, 41924, 42010],
 39: [162, 2984, 3013, 3075, 7880, 19189, 41925],
 40: [8826, 8915, 21469, 21670, 45531, 45

## 字典pt（每个三角面片上随机采样的起始点）

格式如下：

```python
pt = {
    '(tri_index_1, slope_Deg, aspect)': 'coord_x, coord_y',
    ...,
    '(tri_index_n, slope_Deg, aspect)': 'coord_x, coord_y'
}
```
该字典中，键为一个元组，元组中的元素为三角形编号，该三角形的坡度，坡向，代表该随机点所在的三角面片的信息；值是随机点的xy坐标

In [15]:
points = pd.read_csv('D:/edu/研究生数据/30mdata/table_resolutions/30m/30mcenterpoint.csv')
points

,Slope_Pct,Aspect,Tri_Index,p.xy
0,7.453560,116.565051,67694,"316085.22,2899863.82"
1,24.267033,15.945396,68267,"316075.22,2899843.82"
2,24.267033,74.054604,110120,"316085.22,2899833.82"
3,23.570226,98.130102,110254,"316085.22,2899813.82"
4,26.666667,90.000000,68268,"316075.22,2899803.82"
...,...,...,...,...
230305,16.996732,101.309933,36460,"316105.22,2899803.82"
230306,21.081851,71.565051,192060,"316105.22,2899843.82"
230307,23.333333,0.000000,110251,"316055.22,2899833.82"
230308,23.570226,98.130102,110119,"316105.22,2899873.82"


In [16]:
for i in points.index:
    if points.iloc[i,1] < 180:
        points.iloc[i,1] = (points.iloc[i,1] + 180)*math.pi/180
    else:
        points.iloc[i,1] = (points.iloc[i,1] - 180)*math.pi/180
pt = points.groupby(['Tri_Index','Slope_Pct','Aspect']).first()
pt = pt.to_dict()['p.xy']
pt

{(3, 30.73181486, 5.4210152517268835): '321695.22,2903893.82',
 (4, 6.666666667, 3.141592653589793): '314975.22,2905623.82',
 (8, 6.666666667, 0.0): '313495.22,2904943.82',
 (10, 30.18461713, 4.601731759139407): '312965.22,2893763.82',
 (14, 23.33333333, 4.71238898038469): '310475.22,2896233.82',
 (15, 3.333333333, 0.0): '321235.22,2901693.82',
 (18, 44.84541349, 2.303611428654416): '321725.22,2905223.82',
 (22, 32.82952601, 4.2941646508853655): '309955.22,2896213.82',
 (24, 36.05551275, 2.5535900504956586): '321745.22,2905223.82',
 (26, 24.0370085, 2.5535900504956586): '317735.22,2895063.82',
 (28, 22.36067978, 4.248741371332884): '310915.22,2896863.82',
 (34, 40.27681991, 5.139016473853363): '310435.22,2896113.82',
 (38, 7.453559925, 2.0344439361957676): '322265.22,2900523.82',
 (40, 51.85449729, 3.9269908169872414): '313405.22,2893063.82',
 (43, 14.90711985, 4.248741371332884): '312865.22,2906373.82',
 (46, 66.99917081, 1.471127674958227): '321485.22,2904973.82',
 (48, 26.03416559, 

In [17]:
trilist = []
for key in pt:
    trilist.append(key)
# trilist[6]  #随机点所在的三角形的编号，坡度，坡向（反向

In [18]:
trilist

[(3, 30.73181486, 5.4210152517268835),
 (4, 6.666666667, 3.141592653589793),
 (8, 6.666666667, 0.0),
 (10, 30.18461713, 4.601731759139407),
 (14, 23.33333333, 4.71238898038469),
 (15, 3.333333333, 0.0),
 (18, 44.84541349, 2.303611428654416),
 (22, 32.82952601, 4.2941646508853655),
 (24, 36.05551275, 2.5535900504956586),
 (26, 24.0370085, 2.5535900504956586),
 (28, 22.36067978, 4.248741371332884),
 (34, 40.27681991, 5.139016473853363),
 (38, 7.453559925, 2.0344439361957676),
 (40, 51.85449729, 3.9269908169872414),
 (43, 14.90711985, 4.248741371332884),
 (46, 66.99917081, 1.471127674958227),
 (48, 26.03416559, 5.5884470308478855),
 (49, 16.66666667, 0.9272952188017414),
 (50, 40.1386486, 2.2974386668383517),
 (51, 17.95054936, 1.9513027041454667),
 (52, 10.54092553, 1.2490457727983189),
 (53, 19.43650632, 3.682012153824188),
 (54, 93.57112565, 1.6421037919296917),
 (55, 7.453559925, 4.248741371332884),
 (58, 47.72607021, 3.5740004291807175),
 (59, 14.14213562, 5.497787143782138),
 (60, 1

## 字典tridict

通过tirdict来使用三角形的索引编号来定位三角形

In [19]:
tridict = {}
for i in trilist:
    key = i[0]
    value = i
    tridict[key] = value
tridict

{3: (3, 30.73181486, 5.4210152517268835),
 4: (4, 6.666666667, 3.141592653589793),
 8: (8, 6.666666667, 0.0),
 10: (10, 30.18461713, 4.601731759139407),
 14: (14, 23.33333333, 4.71238898038469),
 15: (15, 3.333333333, 0.0),
 18: (18, 44.84541349, 2.303611428654416),
 22: (22, 32.82952601, 4.2941646508853655),
 24: (24, 36.05551275, 2.5535900504956586),
 26: (26, 24.0370085, 2.5535900504956586),
 28: (28, 22.36067978, 4.248741371332884),
 34: (34, 40.27681991, 5.139016473853363),
 38: (38, 7.453559925, 2.0344439361957676),
 40: (40, 51.85449729, 3.9269908169872414),
 43: (43, 14.90711985, 4.248741371332884),
 46: (46, 66.99917081, 1.471127674958227),
 48: (48, 26.03416559, 5.5884470308478855),
 49: (49, 16.66666667, 0.9272952188017414),
 50: (50, 40.1386486, 2.2974386668383517),
 51: (51, 17.95054936, 1.9513027041454667),
 52: (52, 10.54092553, 1.2490457727983189),
 53: (53, 19.43650632, 3.682012153824188),
 54: (54, 93.57112565, 1.6421037919296917),
 55: (55, 7.453559925, 4.24874137133

In [20]:
# tndict[tridict[6]]  #组成三角形的节点的编号以及xyz坐标

In [21]:
# tridict[6]

In [22]:
def point(p):  # 字符串坐标转化为列表形式的浮点坐标的函数
    num = p.count(',')
    if num < 2:   # 当坐标为输入坐标的'x, y, z'，即坐标中包含高程值z时，逗号","的数量为2，当输入的坐标没有z时则逗号的数量为1
        return [float(p.split(',')[0]), float(p.split(',')[1])]
    else:
        return [float(p.split(',')[0]), float(p.split(',')[1]), float(p.split(',')[2])]
# point(pt[tridict[6]])

In [23]:
# 水流抵达公共边时找到水流流入的下一个三角形的函数
def findtri(l1, l2, a):   # l1、l2是公共边两个顶点的所属三角形的列表，a是水流抵达公共边前所在的三角形编号
    i = [item for item in l1 if item in l2]   # 取两个列表的交集，得到共边的两个三角形的编号
    i.remove(a)  # 去掉原先所在的三角形编号，剩下的即为下一个要流入的三角形，若没有下一个三角形则说明水流抵达了边界
    if i ==[]:
        return None
    else:
        return i[0]

In [24]:
def getsum(tri_index):   # 计算编号为tri_index的三角形的三个节点的高程总和
    ttt = tndict[tridict[tri_index]]
    thlist = []
    for value in ttt.values():
        h = point(value)
        thlist.append(h[2])
    return sum(thlist)
# getsum(6)

In [25]:
# 当水流来到三角面片某一节点上，且该节点非局部最高点时，则水流流入三个节点高程总和较高的那一个三角面。
def gettri(n1, n2):   # 输入该节点编号和其周围最高的节点的编号
    l1 = ntdict[n1]
    l2 = ntdict[n2]
    tlist = [item for item in l1 if item in l2]    # 两点确定一条边，确定两个三角面片
    if len(tlist) == 1:
        return tlist[0]
    elif len(tlist) == 2:
        if getsum(tlist[0]) > getsum(tlist[1]):
            return tlist[0]
        else:
            return tlist[1]

In [26]:
def iftop(n_index):   # 当水流来到三角面片的某一节点上时，判断该节点是否为局部最高点
    tri = ntdict[n_index]
    pheight = []
    for i in tri:
        height = tndict[tridict[i]]
        pheight.append(height)
    nid = []
    for i in pheight:
        for key in i.keys():
            if key not in nid:
                nid.append(key)
    xyzlist = []
    for d in pheight:
        for i in d.values():
            pi = point(i)
            if pi not in xyzlist:
                xyzlist.append(pi)
    hlist = []
    for i in xyzlist:
        hlist.append(i[2])
    nh = dict(zip(nid, hlist))
    maxp = []    # 记录了该节点周围高程最高的节点的编号的列表
    for key, value in nh.items():
        if value == max(nh.values()):
            maxp.append(key)
    if n_index in maxp:     # 若该节点在具有最高高程的节点的编号列表里，则该节点视为局部最高点
        return None
    elif len(maxp) == 2:    # 若具有最高高程的节点的编号有两个（不包括输入节点本身），则三点确定一个三角面片。
        l1 = ntdict[maxp[0]]
        l2 = ntdict[maxp[1]]
        tri_index = [item for item in tri if item in l2 and item in l1]
        if tri_index:
            return tri_index[0]
        else:
            return 0.1
    elif len(maxp) ==1:
        tri_index = gettri(n_index, maxp[0])
        return tri_index    # 不是局部最高点则使用gettri函数获取下一个三角面片的编号。

In [27]:
# 判断三点是否一线的函数：ac + bc = ab，则c在ab上，并且返回高程c与高程较高的端点间的距离
def ifinline(a, b, c):   # 输入的a，b，c为[x, y, z]坐标
    ab = round(math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2), 2)
    ac = round(math.sqrt((a[0] - c[0]) ** 2 + (a[1] - c[1]) ** 2), 2)
    bc = round(math.sqrt((b[0] - c[0]) ** 2 + (b[1] - c[1]) ** 2), 2)
    ac_cb = round(ac + bc,2)
    if ac_cb == ab or abs(ac_cb - ab) < 0.04:
        if a[2] > b[2]:
            return [ac, a[2]]
        elif b[2] > a[2]:
            return [bc, b[2]]
        else:
            return None    # 两端点高程相同时返回0
    else:
        return None

In [28]:
# 当水流处于山脊时，水流去往高程较高的一端。
def ridge(p1, p2, p3, p):   # 输入三角形的三个节点
    p1p2p = ifinline(p1, p2 ,p)    # 判断点p在那条边上（三点一线）
    p1p3p = ifinline(p1, p3 ,p)
    p2p3p = ifinline(p2, p3, p)
    rrlist = []
    if p1p2p:
        if p1p2p[0] == 0:  
            return None    # 两端点高高程相等时返回None
        else:
            if p1[2] > p2[2]:
                rrlist.append([p1p2p, p1[:-1]])
            else:
                rrlist.append([p1p2p, p2[:-1]])
    if p1p3p:
        if p1p3p[0] ==0:
            return None
        else:
            if p1[2] > p3[2]:
                rrlist.append([p1p3p, p1[:-1]])
            else:
                rrlist.append([p1p3p, p3[:-1]])
    if p2p3p:
        if p2p3p[0] ==0:
            return None
        else:
            if p2[2] > p3[2]:
                rrlist.append([p2p3p, p2[:-1]])
            else:
                rrlist.append([p2p3p, p3[:-1]])
    if not rrlist:
        return None
    elif len(rrlist) == 2:
        if rrlist[0][0][1] > rrlist[1][0][1]:     #
            length = rrlist[0][0][0]
            nextp = rrlist[0][1]
        else:
            length = rrlist[1][0][0]
            nextp = rrlist[1][1]
    else:
        length = rrlist[0][0][0]
        nextp = rrlist[0][1]
    rr = [length, nextp]   # 返回起始点和山脊上较高的端点的两点间距作为单元坡长，返回下一水流起始点
    return rr

In [29]:
# 坡长递归计算函数
def getslope(p, tri_index):    # 输入水流路径生成起始点和其所在的三角形的编号
    nxy = []      # 顺序记录该三角面片的三个节点的坐标[x, y, z]的列表
    for i in tndict[tridict[tri_index]].values():
        nxy.append(point(i))
    p1 = nxy[0]
    p2 = nxy[1]
    p3 = nxy[2]

    pnode = []    # 顺序记录该三角面片的三个节点的不含高程的坐标[x, y]的列表
    for i in nxy:
        pnode.append(i[:-1])

    nindex = []   # 顺序记录该三角面片的三个节点的编号的列表
    for i in tndict[tridict[tri_index]].keys():
        nindex.append(i)
    n1 = nindex[0]
    n2 = nindex[1]
    n3 = nindex[2]

    if p in pnode:    # 若水流路径生成的起始点在pnode列表里，说明其在三角面片的节点上
        if p == pnode[0]:
            n_index = n1    # 获取节点的编号
        elif p == pnode[1]:
            n_index = n2
        else:
            n_index = n3
        triindex = iftop(n_index)    # 判断该节点是否是局部最高点，若不是则获取并更新下一个三角形的编号，若是则停止递归
        if not triindex:
            return 0
        elif triindex == 0.1:
            tri_indexri = triindex
        else:
            tri_index = triindex   # 更新三角形编号后，重新录入三个节点的坐标和编号。
            nxy = []
            for i in tndict[tridict[tri_index]].values():
                nxy.append(point(i))
            p1 = nxy[0]
            p2 = nxy[1]
            p3 = nxy[2]

            nindex = []
            for i in tndict[tridict[tri_index]].keys():
                nindex.append(i)
            n1 = nindex[0]
            n2 = nindex[1]
            n3 = nindex[2]

    route.append(tri_index)   # 记录水流流经的三角形的编号
    ray = line(sPoint=p, alpha=[tridict[tri_index][2]])   # 输入起始点p和三角形的坡向生成水流射线
    l1 = dotPair(sPoint=p1, ePoint=p2)    # 三角形的三个节点分别形成三条线段
    l2 = dotPair(sPoint=p1, ePoint=p3)
    l3 = dotPair(sPoint=p2, ePoint=p3)
    if getDistance(ray, l1) is not None:
        length = getDistance(ray, l1)[1]
        p = getDistance(ray, l1)[0]
        tri_index = findtri(ntdict[n1], ntdict[n2], tri_index)
    elif getDistance(ray, l2) is not None:
        length = getDistance(ray, l2)[1]
        p = getDistance(ray, l2)[0]
        tri_index = findtri(ntdict[n1], ntdict[n3], tri_index)
    elif getDistance(ray, l3) is not None:    # 三个if顺序判断水流射线与三条线段是否有交点，若有交点则计算单元坡长和交点坐标。
        length = getDistance(ray, l3)[1]
        p = getDistance(ray, l3)[0]    # 将交点作为下一个水流射线的起始点
        tri_index = findtri(ntdict[n2], ntdict[n3], tri_index)    # 水流抵达公共边时找到水流流入的下一个三角形
    else:     # 若水流射线与三条边都没有交点，说明该边为山脊
        pinridge = ridge(p1, p2, p3, p)
        if not pinridge:
            return 0
        else:
            length = pinridge[0]
            p = pinridge[1]
    routep.append(p)
    if tri_index is None:
        return 0
    else:
        slopel = length + getslope(p, tri_index)   # 坡长累加
    return slopel

In [30]:
# route = []
# p = point(pt[tridict[6]])
# routep = [p]
# result = getslope(p, 6)
# print(result)
# print(route)

In [31]:
with open('D:/edu/研究生数据/30mdata/table_resolutions/30m/30mslopelength.csv', 'w', encoding='UTF8', newline='') as f:
    header = ['tri_id', 'slopelength', 'routep']
    writer = csv.writer(f)
    writer.writerow(header)
    for i in tqdm(tridict):
        p = point(pt[tridict[i]])
        routep = [p]
        route = []
        result = getslope(p, i)
        data = [i, result, routep]
        writer.writerow(data)

100%|█████████████████████████████████████████████████████████████████████████| 230310/230310 [07:09<00:00, 535.79it/s]


In [3]:
np.linalg.det([[1,2],[3,4]])

-2.0000000000000004